In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
import torch.distributed as dist
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
!pip install wandb
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.


wandb: Currently logged in as: wwblodge1 (use `wandb login --relogin` to force relogin)


True

In [5]:
wandb.init(project="hw9_instance")

In [6]:
# Assume that this notebook only sees one GPU.
GPU=0

In [7]:
SEED=1

In [8]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [9]:
torch.cuda.device_count()

1

In [10]:
START_EPOCH = 0

In [11]:
ARCH = 'resnet18'
EPOCHS = 1
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
TRAINDIR="data/train"
VALDIR="data/val"

global_step = 0


In [12]:
TRAINDIR="data/train"
VALDIR="data/val"

In [13]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [14]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [15]:
# how many processes in cluster?
WORLD_SIZE = 2
BACKEND = 'nccl'
# where is the master?
# export NCCL_SOCKET_IFNAME=172.31.24.47
URL = 'tcp://172.31.24.47:1234'

In [16]:
#what is my rank?
RANK = 0

In [17]:
dist.init_process_group(backend = BACKEND, init_method= URL,
                                world_size= WORLD_SIZE, rank=RANK)

In [18]:
#torch.cuda.set_device('cpu')
torch.cuda.set_device(GPU)

In [19]:
cudnn.benchmark = True

In [20]:
# def train(train_loader, model, criterion, optimizer, epoch):
#     batch_time = AverageMeter('Time', ':6.3f')
#     data_time = AverageMeter('Data', ':6.3f')
#     losses = AverageMeter('Loss', ':.4e')
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     top5 = AverageMeter('Acc@5', ':6.2f')
#     progress = ProgressMeter(
#         len(train_loader),
#         [batch_time, data_time, losses, top1, top5],
#         prefix="Epoch: [{}]".format(epoch))

#     # switch to train mode
#     model.train()

#     end = time.time()
#     for i, (images, target) in enumerate(train_loader):
#         # measure data loading time
#         data_time.update(time.time() - end)

#         if GPU is not None:
#             images = images.cuda(GPU, non_blocking=True)
#         if torch.cuda.is_available():
#             target = target.cuda(GPU, non_blocking=True)

#         # compute output
#         output = model(images)
#         loss = criterion(output, target)

#         # measure accuracy and record loss
#         acc1, acc5 = accuracy(output, target, topk=(1, 5))
#         losses.update(loss.item(), images.size(0))
#         top1.update(acc1[0], images.size(0))
#         top5.update(acc5[0], images.size(0))

#         # compute gradient and do SGD step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if i % PRINT_FREQ == 0:
#             progress.display(i)

def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [21]:
# def validate(val_loader, model, criterion):
#     batch_time = AverageMeter('Time', ':6.3f')
#     losses = AverageMeter('Loss', ':.4e')
#     top1 = AverageMeter('Acc@1', ':6.2f')
#     top5 = AverageMeter('Acc@5', ':6.2f')
#     progress = ProgressMeter(
#         len(val_loader),
#         [batch_time, losses, top1, top5],
#         prefix='Test: ')

#     # switch to evaluate mode
#     model.eval()

#     with torch.no_grad():
#         end = time.time()
#         for i, (images, target) in enumerate(val_loader):
#             if GPU is not None:
#                 images = images.cuda(GPU, non_blocking=True)
#             if torch.cuda.is_available():
#                 target = target.cuda(GPU, non_blocking=True)

#             # compute output
#             output = model(images)
#             loss = criterion(output, target)

#             # measure accuracy and record loss
#             acc1, acc5 = accuracy(output, target, topk=(1, 5))
#             losses.update(loss.item(), images.size(0))
#             top1.update(acc1[0], images.size(0))
#             top5.update(acc5[0], images.size(0))

#             # measure elapsed time
#             batch_time.update(time.time() - end)
#             end = time.time()

#             if i % PRINT_FREQ == 0:
#                 progress.display(i)

#         # TODO: this should also be done with the ProgressMeter
#         print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
#               .format(top1=top1, top5=top5))

#     return top1.avg

def validate(val_loader, model, criterion):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})

    return top1.avg


In [22]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [24]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [25]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [26]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [27]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [28]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [29]:
# IMG_SIZE = 32
IMG_SIZE = 224

In [30]:
NUM_CLASSES = 1000

In [31]:
model = models.__dict__[ARCH]()

In [32]:
inf = model.fc.in_features

In [33]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [34]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])
# model = torch.nn.parallel.DistributedDataParallel(model)

In [35]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [36]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [37]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [38]:
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [39]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [40]:
# transform_val = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(cinic_mean_RGB, cinic_std_RGB),
# ])
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [41]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [42]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [43]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [44]:
best_acc1 = 0

In [45]:
%%time
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 11.614 (11.614)	Data  4.333 ( 4.333)	Loss 7.0065e+00 (7.0065e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.60 (  0.60)
Epoch: [0][  10/1282]	Time  0.839 ( 2.231)	Data  0.105 ( 0.893)	Loss 6.9203e+00 (6.9694e+00)	Acc@1   0.60 (  0.22)	Acc@5   1.80 (  0.71)
Epoch: [0][  20/1282]	Time  0.821 ( 2.046)	Data  0.089 ( 0.986)	Loss 6.9500e+00 (6.9570e+00)	Acc@1   0.40 (  0.20)	Acc@5   1.20 (  0.81)
Epoch: [0][  30/1282]	Time  0.828 ( 1.998)	Data  0.091 ( 1.035)	Loss 6.7850e+00 (6.9201e+00)	Acc@1   0.80 (  0.28)	Acc@5   1.80 (  1.11)
Epoch: [0][  40/1282]	Time  0.839 ( 1.976)	Data  0.095 ( 1.062)	Loss 6.7368e+00 (6.8854e+00)	Acc@1   0.80 (  0.39)	Acc@5   3.40 (  1.41)
Epoch: [0][  50/1282]	Time  0.751 ( 1.970)	Data  0.002 ( 1.086)	Loss 6.6988e+00 (6.8418e+00)	Acc@1   0.00 (  0.43)	Acc@5   1.40 (  1.67)
Epoch: [0][  60/1282]	Time  0.755 ( 1.963)	Data  0.002 ( 1.099)	Loss 6.5351e+00 (6.7995e+00)	Acc@1   1.20 (  0.51)	Acc@5   4.00 (  1.94)
Epoch: [0][  70/1282]	Time  1.279 ( 1.957

Epoch: [0][ 600/1282]	Time  0.737 ( 2.614)	Data  0.003 ( 0.167)	Loss 4.6482e+00 (5.4323e+00)	Acc@1  11.20 (  6.45)	Acc@5  29.60 ( 17.42)
Epoch: [0][ 610/1282]	Time  0.729 ( 2.614)	Data  0.002 ( 0.165)	Loss 4.4907e+00 (5.4163e+00)	Acc@1  14.60 (  6.58)	Acc@5  31.40 ( 17.68)
Epoch: [0][ 620/1282]	Time  0.726 ( 2.616)	Data  0.002 ( 0.162)	Loss 4.4576e+00 (5.4010e+00)	Acc@1  12.80 (  6.70)	Acc@5  35.80 ( 17.94)
Epoch: [0][ 630/1282]	Time  0.733 ( 2.619)	Data  0.003 ( 0.160)	Loss 4.4590e+00 (5.3858e+00)	Acc@1  17.20 (  6.82)	Acc@5  34.40 ( 18.18)
Epoch: [0][ 640/1282]	Time  0.762 ( 2.619)	Data  0.002 ( 0.158)	Loss 4.5057e+00 (5.3715e+00)	Acc@1  12.80 (  6.94)	Acc@5  33.20 ( 18.41)
Epoch: [0][ 650/1282]	Time  0.752 ( 2.619)	Data  0.002 ( 0.155)	Loss 4.4400e+00 (5.3564e+00)	Acc@1  15.00 (  7.05)	Acc@5  33.80 ( 18.65)
Epoch: [0][ 660/1282]	Time  0.755 ( 2.627)	Data  0.002 ( 0.153)	Loss 4.4828e+00 (5.3417e+00)	Acc@1  11.40 (  7.17)	Acc@5  30.80 ( 18.90)
Epoch: [0][ 670/1282]	Time  0.754 ( 2.630

Epoch: [0][1200/1282]	Time  0.722 ( 2.760)	Data  0.002 ( 0.085)	Loss 3.4807e+00 (4.7241e+00)	Acc@1  28.40 ( 13.03)	Acc@5  53.60 ( 29.53)
Epoch: [0][1210/1282]	Time  0.719 ( 2.759)	Data  0.002 ( 0.085)	Loss 3.5992e+00 (4.7146e+00)	Acc@1  24.40 ( 13.13)	Acc@5  48.20 ( 29.70)
Epoch: [0][1220/1282]	Time  0.732 ( 2.760)	Data  0.002 ( 0.084)	Loss 3.5245e+00 (4.7053e+00)	Acc@1  27.80 ( 13.24)	Acc@5  48.00 ( 29.87)
Epoch: [0][1230/1282]	Time  0.736 ( 2.761)	Data  0.003 ( 0.083)	Loss 3.6226e+00 (4.6962e+00)	Acc@1  24.20 ( 13.33)	Acc@5  49.60 ( 30.03)
Epoch: [0][1240/1282]	Time  0.730 ( 2.759)	Data  0.002 ( 0.083)	Loss 3.7131e+00 (4.6869e+00)	Acc@1  24.80 ( 13.44)	Acc@5  47.60 ( 30.20)
Epoch: [0][1250/1282]	Time  0.745 ( 2.759)	Data  0.002 ( 0.082)	Loss 3.5407e+00 (4.6777e+00)	Acc@1  26.00 ( 13.54)	Acc@5  51.40 ( 30.36)
Epoch: [0][1260/1282]	Time  0.753 ( 2.759)	Data  0.002 ( 0.082)	Loss 3.3935e+00 (4.6687e+00)	Acc@1  27.60 ( 13.64)	Acc@5  54.40 ( 30.52)
Epoch: [0][1270/1282]	Time  0.734 ( 2.757

Epoch: [1][ 500/1282]	Time  0.751 ( 2.795)	Data  0.002 ( 0.025)	Loss 3.6589e+00 (3.5291e+00)	Acc@1  27.00 ( 26.35)	Acc@5  50.60 ( 50.87)
Epoch: [1][ 510/1282]	Time  0.755 ( 2.797)	Data  0.003 ( 0.025)	Loss 3.5254e+00 (3.5291e+00)	Acc@1  25.40 ( 26.34)	Acc@5  52.20 ( 50.86)
Epoch: [1][ 520/1282]	Time  0.754 ( 2.799)	Data  0.003 ( 0.024)	Loss 3.4425e+00 (3.5283e+00)	Acc@1  27.80 ( 26.37)	Acc@5  53.60 ( 50.87)
Epoch: [1][ 530/1282]	Time  0.760 ( 2.800)	Data  0.002 ( 0.024)	Loss 3.4030e+00 (3.5283e+00)	Acc@1  28.60 ( 26.38)	Acc@5  52.40 ( 50.88)
Epoch: [1][ 540/1282]	Time  0.754 ( 2.802)	Data  0.002 ( 0.023)	Loss 3.4854e+00 (3.5280e+00)	Acc@1  27.60 ( 26.37)	Acc@5  53.00 ( 50.89)
Epoch: [1][ 550/1282]	Time  0.762 ( 2.804)	Data  0.003 ( 0.023)	Loss 3.6626e+00 (3.5281e+00)	Acc@1  25.60 ( 26.37)	Acc@5  49.20 ( 50.90)
Epoch: [1][ 560/1282]	Time  0.762 ( 2.804)	Data  0.003 ( 0.023)	Loss 3.5031e+00 (3.5280e+00)	Acc@1  25.80 ( 26.38)	Acc@5  51.00 ( 50.89)
Epoch: [1][ 570/1282]	Time  0.764 ( 2.800

Epoch: [1][1100/1282]	Time  0.767 ( 2.838)	Data  0.002 ( 0.013)	Loss 3.3363e+00 (3.5058e+00)	Acc@1  26.40 ( 26.59)	Acc@5  54.80 ( 51.29)
Epoch: [1][1110/1282]	Time  0.761 ( 2.834)	Data  0.002 ( 0.013)	Loss 3.3862e+00 (3.5046e+00)	Acc@1  30.80 ( 26.61)	Acc@5  52.20 ( 51.30)
Epoch: [1][1120/1282]	Time  0.769 ( 2.834)	Data  0.003 ( 0.013)	Loss 3.3897e+00 (3.5037e+00)	Acc@1  29.80 ( 26.63)	Acc@5  52.00 ( 51.32)
Epoch: [1][1130/1282]	Time  0.795 ( 2.833)	Data  0.002 ( 0.013)	Loss 3.2934e+00 (3.5030e+00)	Acc@1  31.20 ( 26.64)	Acc@5  56.60 ( 51.33)
Epoch: [1][1140/1282]	Time  0.801 ( 2.830)	Data  0.003 ( 0.013)	Loss 3.4520e+00 (3.5023e+00)	Acc@1  25.60 ( 26.65)	Acc@5  53.60 ( 51.35)
Epoch: [1][1150/1282]	Time  0.766 ( 2.829)	Data  0.002 ( 0.013)	Loss 3.3893e+00 (3.5014e+00)	Acc@1  25.20 ( 26.65)	Acc@5  54.60 ( 51.37)
Epoch: [1][1160/1282]	Time  0.770 ( 2.827)	Data  0.003 ( 0.013)	Loss 3.3760e+00 (3.5000e+00)	Acc@1  28.80 ( 26.67)	Acc@5  50.40 ( 51.39)
Epoch: [1][1170/1282]	Time  0.764 ( 2.829

In [4]:

writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 25910), started 0:35:09 ago. (Use '!kill 25910' to kill it.)

In [4]:
%load_ext tensorboard
%tensorboard --logdir=runs